In [35]:
import pandas as pd
import os
import numpy as np
from dataclasses import dataclass
import random
import time
import glob

np.random.seed(14)

In [36]:
@dataclass
class Particle:
    x: float
    y: float
    theta: float
    weight: float 
        


@dataclass
class LandMark:
    x: float
    y: float
    index: int

In [78]:
gt_data = pd.read_csv('data/gt_data.txt', names=['X','Y','Orientation'], sep=' ')
map_data = pd.read_csv('data/map_data.txt', names=['X','Y','# landmark'])
control_data = pd.read_csv('data/control_data.txt', names=['velocity','Yaw rate'], sep=' ')

#observation = pd.read_csv('data/observation/observations_000001.txt', names = ['X cord','Y cord'], sep=' ')


result = [(x,y, landmark) for x,y,landmark in zip(map_data['X'],map_data['Y'], map_data['# landmark'])]
landarkList=[]
for res in result:
    l = LandMark(res[0],res[1],res[2])
    landarkList.append(l)
    
obs_path = glob.glob("data/SensorDataFiles/observation*.txt")
obs_path.sort()

print('Loading the Observations..')
observation=[]
for file_path in obs_path:
    observationTmp = pd.read_csv(file_path, names = ['X cord','Y cord'], sep=' ')
    observation.append(observationTmp)
print('Loading Done!')

Loading the Observations..
Loading Done!


In [89]:
def calculateDistance(landmark1, landmark2):
    a =  np.sqrt((landmark1.x - landmark2.x)**2 + (landmark1.y - landmark2.y)**2)
    return a
    
 
    
def findClosestLandmark(map_landmarks, singleObs): # edited
    map_landmarks_arr = np.array([[lm.x,lm.y] for lm in map_landmarks])
    singleObs_arr = np.array([[singleObs.x,singleObs.y]])
    min_index = np.argmin(np.sqrt(np.square((map_landmarks_arr-singleObs_arr)).sum(1)))
    closest_landmark = map_landmarks[min_index]
    
    return closest_landmark
    
    
def getError(gt_data, bestParticle):
    error1 = np.abs(gt_data[0] - bestParticle.x)
    error2 = np.abs(gt_data[1] - bestParticle.y)
    error3 = np.abs(gt_data[2] - bestParticle.theta)
    if(error3>2*np.pi):
        error3 = 2*np.pi - error3
    return (error1, error2, error3)

def findObservationProbability(closest_landmark,map_coordinates, sigmaX, sigmaY):
    
    mew_x = closest_landmark.x
    mew_y = closest_landmark.y
    
    x = map_coordinates.x;
    y = map_coordinates.y;
    denom = 1 / np.sqrt(sigmaX * sigmaY * 2 * np.pi)
    frac =  (1/ (2 * np.pi * sigmaX * sigmaY ))
    weight1 = np.square((x - mew_x) / sigmaX) + np.square((y - mew_y) / sigmaY)
    ans = np.exp(-0.5*weight1)/denom
    return max(ans, np.finfo('float').eps)



def mapObservationToMapCoordinates(observation, particle):
    x = observation.x
    y = observation.y

    xt = particle.x
    yt = particle.y
    theta = particle.theta

    MapX = x * np.cos(theta) - y * np.sin(theta) + xt
    MapY = x * np.sin(theta) + y * np.cos(theta) + yt

    return MapX, MapY

def mapObservationsToMapCordinatesList(observations, particle):
    
    convertedObservations=[]
    i=0
    for obs in observations.iterrows():
        singleObs = LandMark(obs[1][0],obs[1][1],1)
        mapX, mapY = mapObservationToMapCoordinates(singleObs, particle)
        tmpLandmark = LandMark(x=mapX, y=mapY, index=i)
        i+=1
        convertedObservations.append(tmpLandmark)
    return convertedObservations



In [90]:
class ParticleFilter:
    particles = []
    def __init__(self, intialX, initialY, std, numOfParticles):
        self.number_of_particles = numOfParticles
        for i in range(self.number_of_particles):
           # tmpParticle = Particle(intialX,initialY,0,1)
            x = random.gauss(intialX, std)
            y = random.gauss(initialY, std)
            theta = random.uniform(0, 2*np.pi)
            tmpParticle = Particle(x,y , theta,1)
            self.particles.append(tmpParticle)
            
            
            
    def moveParticles(self, velocity, yaw_rate, delta_t=0.1):
        
        for i in range(self.number_of_particles):
            if(yaw_rate!=0):
                theta = self.particles[i].theta
                newTheta = theta + delta_t*yaw_rate;
                newX =  self.particles[i].x +(velocity/yaw_rate)*(np.sin(newTheta)-np.sin(theta));
                newY =  self.particles[i].y +(velocity/yaw_rate)*(np.cos(theta)-np.cos(newTheta));
                
                #todo Add noise!!
                self.particles[i].x = newX +random.gauss(0, 0.3)
                self.particles[i].y = newY +random.gauss(0, 0.3)
                self.particles[i].theta = newTheta +random.gauss(0, 0.01)
            else:
                print("ZERO!!!")
            

        
        
    def UpdateWeight(self, observations): #edited
        for i,particle in enumerate(self.particles):
            glob_obs = mapObservationsToMapCordinatesList(observations,particle)
            weight = 1
            
            for glob_ob_single in glob_obs:
                closest_lm = findClosestLandmark(landarkList,glob_ob_single)
                obs_prob = findObservationProbability(closest_lm,glob_ob_single,sigmaY,sigmaY)
                weight *= obs_prob
            self.particles[i].weight = weight
            
    
    
    def getBestParticle(self):
        best_particle =  max(self.particles, key= lambda particle: particle.weight)
        return best_particle    
    
    def getBestParticleOut(self):
        x=0
        y=0
        theta=0
        for i in range(self.number_of_particles):
            x+= self.particles[i].x
            y+= self.particles[i].y
            theta+= self.particles[i].theta
        x=x/self.number_of_particles
        y=y/self.number_of_particles
        theta=theta/self.number_of_particles
        best_particle =  Particle(x,y,theta, weight=1)
        return best_particle        
    
    def PrintWeights(self):
        for i in range(self.number_of_particles):
            print("Weight:",self.particles[i].weight, self.particles[i].x,self.particles[i].y)
    
    
    def Resample(self): # edited
        w = np.array([x.weight for x in self.particles])
        w = w / w.sum()
        max_w_2 = w.max() * 2

        curr_ind = np.random.randint(0, self.number_of_particles)
        new_particles_lst = []
        for i in range(self.number_of_particles):
            offset = np.random.random() * max_w_2

            while offset > w[curr_ind]:
                offset -= w[curr_ind]
                curr_ind = (curr_ind + 1) % self.number_of_particles

            new_particle = Particle(
                self.particles[curr_ind].x,
                self.particles[curr_ind].y,
                self.particles[curr_ind].theta,
                self.particles[curr_ind].weight
            )
            new_particles_lst.append(new_particle)

        self.particles = list(new_particles_lst)
        
       
        
        

In [91]:
sigmaY = 0.3

magicNumberOfParticles = 200


start = time.time()

def main():
    particleFilter = ParticleFilter(0 ,0 ,sigmaY, numOfParticles=magicNumberOfParticles)
    #articleFilter = ParticleFilter(6.2785 ,1.9598 ,sigmaY, numOfParticles=magicNumberOfParticles)


    for i,obs in enumerate(observation):
        #prediction
        if(i!=0):
            velocity = control_data.iloc[i-1][0]
            yaw_rate = control_data.iloc[i-1][1]
            particleFilter.moveParticles(velocity, yaw_rate)
        a = obs.copy()
        particleFilter.UpdateWeight(a)
        particleFilter.Resample()
        bestP = particleFilter.getBestParticle()
        error = getError(gt_data.iloc[i], bestP)
        print(i,error)

    end = time.time()
    print("Time: {:.3f} secs".format(end - start))
        
main()

0 (6.683632892177622, 2.0305696772657056, 3.6908939992944103)
1 (6.202104588200858, 2.037602764499923, 0.2475383472768347)
2 (5.784668829500402, 2.225635426671559, 0.24826137035111612)
3 (5.204638522083325, 1.7149174519406722, 0.21603172953633304)
4 (4.812309246505658, 2.051459455349272, 0.20671938438251508)
5 (4.43209843956164, 1.532870282126318, 0.20222004705092717)
6 (4.2824551843649115, 1.5155106606061926, 0.18784762896835555)
7 (3.899758648553438, 1.285290882683236, 0.17037511202211947)
8 (3.402933346448134, 1.1858173709638746, 0.16248510137533606)
9 (2.930089153153477, 1.0311829040988316, 0.1495438741282295)
10 (2.6153529221689062, 0.9300919382614468, 0.13452946850523745)
11 (2.543292934332161, 1.189209480838369, 0.10928487792845593)
12 (2.16225594448116, 1.178012463287724, 0.07526454373923908)
13 (1.970212261071957, 0.7028051385515148, 0.05899534610595647)
14 (1.5096882345712537, 0.516653589465379, 0.04149632051498464)
15 (0.8209717527426932, 0.6038001074388357, 0.02416240536856

KeyboardInterrupt: 

In [40]:
import cv2

In [ ]:
# WIDTH=1600
# HEIGHT=1200

# def mouseCallback(event, x, y, flags,null):
#     global i
    

# WINDOW_NAME="Particle Filter"
# img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
# cv2.namedWindow(WINDOW_NAME)
# DELAY_MSEC=50
# #cv2.setMouseCallback(WINDOW_NAME,mouseCallback)

# i=1


    

# particleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=50)

# while(1):
 
#     cv2.imshow(WINDOW_NAME,img)
#     img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
    
#     if(i>2000):
#         break
    
#     for landmark in landarkList:
#         cv2.circle(img,tuple((int(landmark.x)*2+400, int(landmark.y)*2+300)),3,(255,0,0),-1)
    
#     #draw_particles:
#     for particle in particleFilter.particles:
#         cv2.circle(img,tuple((int(particle.x)*2+400,int(particle.y)*2+300)),1,(255,255,255),-1)
#     cv2.circle(img,tuple((int(gt_data.iloc[i].X)*2+400,int(gt_data.iloc[i].Y)*2+300)),1,(0,255,0),-1)
    
#     if cv2.waitKey(DELAY_MSEC) & 0xFF == 27:
#         break
#     if cv2.waitKey(33) == ord('a'):
#         velocity = control_data.iloc[i-1][0]
#         yaw_rate = control_data.iloc[i-1][1]
#         particleFilter.moveParticles(velocity, yaw_rate)
#         a = observation[i].copy()
#         particleFilter.UpdateWeight(a)
#         particleFilter.PrintWeights()
#         particleFilter.Resample()
        
#         i+=1

# cv2.destroyAllWindows()
